In [5]:
with open(r'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\zuco1_normalreading_ner.txt', 'r') as file:
    # Iterate through each line
    for line in file:
        # Split the line by the tab character
        parts = line.split('\t')

        # Check if there are at least two parts
        if len(parts) >= 2:
            word = parts[0]
            class_ = parts[1].strip()  # strip() to remove leading/trailing whitespace

            # Do something with the word and class, for example, print them
            print("Word:", word)
            print("Class:", class_)
        else:
            # Handle the case where there is not enough data in the line
            print("Invalid line:", line.strip())

Word: Henry
Class: B-PER
Word: Ford
Class: I-PER
Word: advocated
Class: O
Word: long-time
Class: O
Word: associate
Class: O
Word: Harry
Class: B-PER
Word: Bennett
Class: I-PER
Word: to
Class: O
Word: take
Class: O
Word: the
Class: O
Word: spot
Class: O
Word: .
Class: O
Invalid line: 
Word: Edsel
Class: B-PER
Word: 's
Class: O
Word: widow
Class: O
Word: Eleanor
Class: B-PER
Word: ,
Class: O
Word: who
Class: O
Word: had
Class: O
Word: inherited
Class: O
Word: Edsel
Class: B-PER
Word: 's
Class: O
Word: voting
Class: O
Word: stock
Class: O
Word: ,
Class: O
Word: wanted
Class: O
Word: her
Class: O
Word: son
Class: O
Word: Henry
Class: B-PER
Word: Ford
Class: I-PER
Word: II
Class: I-PER
Word: to
Class: O
Word: take
Class: O
Word: over
Class: O
Word: the
Class: O
Word: position
Class: O
Word: .
Class: O
Invalid line: 
Word: Aldous
Class: B-PER
Word: Leonard
Class: I-PER
Word: Huxley
Class: I-PER
Word: (
Class: O
Word: July
Class: O
Word: 26
Class: O
Word: ,
Class: O
Word: 1894
Class: O
Word: 